In [1]:
# Import packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.metrics import accuracy_score, f1_score, make_scorer, classification_report
from sklearn.utils import shuffle
from imblearn.over_sampling import SMOTENC
import matplotlib.pyplot as plt
import mord
from collections import Counter

Using TensorFlow backend.


In [2]:
path = "/Users/Stijn/Documents/Master Data Science and Society/Block 3/thesis/code/thesis_Code/"
mood = pd.read_csv(path+'mood_imputed_median.csv', sep = ',', index_col=0)
mood['bored_last'] = mood.groupby('user_id')['bored'].shift()
mood.loc[(pd.isnull(mood.bored_last)), 'bored_last'] = mood['bored']
mood_bored = mood.drop(["anxious", "content", "cheerful", "user_id", "response_time"], axis=1)

In [3]:
mood_bored[:5]

,bored,day_time_window,average_TimeUse,bulk,messaging,socialnetworking,otherapp,bored_last
4825,3.0,3.0,0.673000,0,0,0,1,3.0
4826,3.0,4.0,64.314667,0,1,0,2,3.0
245,3.0,1.0,20.590500,0,0,0,2,3.0
4827,3.0,2.0,0.000000,0,0,0,0,3.0
4828,3.0,3.0,53.957200,0,3,0,2,3.0


In [4]:
# Normalize predictors
mood_normz = mood_bored.copy()
features_to_normalize = ['day_time_window', 'average_TimeUse', 'messaging', 'socialnetworking', 'otherapp']
mood_normz[features_to_normalize] = mood_normz[features_to_normalize].apply(lambda x:(x-x.min()) / (x.max()-x.min()))

In [46]:
# X = predictors, y = target
X = mood_normz.iloc[:,1:]
y = mood_normz.iloc[:, 0:1]

# Convert float to int
y = y.astype(int)

In [47]:
# Train (80%) & Test set (20%)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [48]:
# Oversampling unbalanced target
print("Before OverSampling, counts of labels: {}".format(y['bored'].value_counts()))
  
sm = SMOTENC(categorical_features=[0, 2, 6], random_state = 2) 
X_train_res, y_train_res = sm.fit_sample(X, y['bored'].ravel()) 

print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0))) 
print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1))) 
print("After OverSampling, counts of label '2': {}".format(sum(y_train_res == 2))) 
print("After OverSampling, counts of label '3': {}".format(sum(y_train_res == 3))) 
print("After OverSampling, counts of label '4': {}".format(sum(y_train_res == 4))) 
print("After OverSampling, counts of label '5': {}".format(sum(y_train_res == 5))) 

Before OverSampling, counts of labels: 0    6403
1    2418
2    2295
3    1485
4     522
5      98
Name: bored, dtype: int64
After OverSampling, counts of label '0': 6403
After OverSampling, counts of label '1': 6403
After OverSampling, counts of label '2': 6403
After OverSampling, counts of label '3': 6403
After OverSampling, counts of label '4': 6403
After OverSampling, counts of label '5': 6403


In [98]:
# Convert oversample dataset to Pandas DataFrame
X_train = pd.DataFrame(data=X_train_res)
y_train = pd.DataFrame(data=y_train_res)

In [99]:
X_train_nomood = X_train.iloc[:,:-1]
X_train_onlyboredlast = X_train.iloc[:,-1:]
# X_test_nomood = X_test.iloc[:,:-1]
# X_test_onlyboredlast = X_test.iloc[:,-1:]

In [100]:
# X_test_bored = X_test_boredseries.values

X_train = X_train_nomood.values
y_train = y_train.values.ravel()

In [101]:
len(y_train)

38418

In [102]:
# Shuffle both train and test set to cancel patterns
X, y = shuffle(X_train, y_train)
#X_te, y_te = shuffle(X_test_nomood, y_test)

In [103]:
parameter_candidates = {'max_iter': [10, 100, 250, 500], 'alpha': [0.01, 0.1, 0.5, 1.0]}
clf = GridSearchCV(estimator=mord.LogisticIT(), 
                    param_grid=parameter_candidates,
                    cv=10,
                   scoring = 'neg_mean_squared_error',
                    n_jobs=-1)

In [104]:
clf_fit = clf.fit(X, y)

In [105]:
print('Best score:', clf_fit.best_score_) 
print(" ")
print('Best estimator:',clf_fit.best_estimator_) 

Best score: -7.8686813472851265
 
Best estimator: LogisticIT(alpha=0.01, max_iter=100, verbose=0)


In [10]:
def macroaverage_mae(test, pred):
    mae_0, mae_1, mae_2, mae_3, mae_4, mae_5 = (0,0,0,0,0,0)

    for i in range(len(pred)):
        if test[i] == 0 and pred[i] != 0:
            mae_0 += (abs(test[i]-pred[i]))
        if test[i] == 1 and pred[i] != 1:
            mae_1 += (abs(test[i]-pred[i]))
        if test[i] == 2 and pred[i] != 2:
            mae_2 += (abs(test[i]-pred[i]))
        if test[i] == 3 and pred[i] != 3:
            mae_3 += (abs(test[i]-pred[i]))
        if test[i] == 4 and pred[i] != 4:
            mae_4 += (abs(test[i]-pred[i]))
        if test[i] == 5 and pred[i] != 5:
            mae_5 += (abs(test[i]-pred[i]))

    cnt_0, cnt_1, cnt_2, cnt_3, cnt_4, cnt_5 = (0,0,0,0,0,0)
    for z in range(len(test)):
        if test[z] == 0:
            cnt_0 += 1
        if test[z] == 1:
            cnt_1 += 1
        if test[z] == 2:
            cnt_2 += 1
        if test[z] == 3:
            cnt_3 += 1
        if test[z] == 4:
            cnt_4 += 1
        if test[z] == 5:
            cnt_5 += 1

    mae_macroaverage = ((mae_0/cnt_0) + (mae_1/cnt_1) + (mae_2/cnt_2) + (mae_3/cnt_3) + (mae_4/cnt_4) + (mae_5/cnt_5)) / 6
    return mae_macroaverage 

#score = make_scorer(macroaverage_mae, greater_is_better=False)

In [14]:
# X = predictors, y = target
X = mood_normz.iloc[:,1:-1]
y = mood_normz.iloc[:, 0:1]

# Convert float to int
y = y.astype(int)

In [15]:
X_train, y_train = shuffle(X, y)

In [16]:
# Convert oversample dataset to Pandas DataFrame
X = pd.DataFrame(data=X_train)
y = pd.DataFrame(data=y_train)

In [17]:
n_split=10
f1_scores = []
f1_scores_classes = []
mmae_scores = []
for train_index, test_index in KFold(n_splits = n_split, random_state=2, shuffle=True).split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    sm = SMOTENC(categorical_features=[0, 2], random_state = 2) 
    X_train_oversampl, y_train_oversampl = sm.fit_sample(X_train, y_train['bored'].ravel()) 
    X_use, y_use = shuffle(X_train_oversampl, y_train_oversampl)
    
    model = mord.LogisticIT(alpha=0.1, max_iter=100, verbose=0)
    model.fit(X_use, y_use)   
    pred = model.predict(X_test)
    print("Predicted values: ", Counter(pred))
    
    f1_scores.append(round(f1_score(y_test, pred, average='weighted'), 4))
    f1_scores_classes.append(f1_score(y_test, pred, average=None))
    
    test = y_test.values
    mmae_scores.append(macroaverage_mae(test, pred))

Predicted values:  Counter({0: 676, 5: 600, 4: 27, 2: 11, 3: 6, 1: 3})
Predicted values:  Counter({0: 703, 5: 591, 4: 11, 2: 7, 3: 6, 1: 4})
Predicted values:  Counter({0: 699, 5: 581, 4: 20, 1: 10, 2: 9, 3: 3})
Predicted values:  Counter({0: 674, 5: 592, 4: 36, 3: 7, 2: 7, 1: 6})
Predicted values:  Counter({0: 724, 5: 569, 4: 21, 1: 3, 3: 3, 2: 2})
Predicted values:  Counter({0: 729, 5: 557, 4: 25, 1: 5, 3: 4, 2: 2})
Predicted values:  Counter({0: 706, 5: 569, 4: 21, 3: 11, 2: 9, 1: 6})
Predicted values:  Counter({0: 726, 5: 566, 4: 16, 1: 7, 2: 6, 3: 1})
Predicted values:  Counter({0: 698, 5: 584, 4: 22, 3: 8, 2: 7, 1: 3})
Predicted values:  Counter({0: 687, 5: 596, 4: 24, 1: 7, 2: 6, 3: 2})


In [18]:
print("Mean F1-score per class: ", [round(np.mean(x), 4) for x in zip(*f1_scores_classes)])
print("STD F1-score per class: ", [round(np.std(x), 4) for x in zip(*f1_scores_classes)])

Mean F1-score per class:  [0.5259, 0.0109, 0.0093, 0.0074, 0.0212, 0.0196]
STD F1-score per class:  [0.0165, 0.0099, 0.0078, 0.0099, 0.0262, 0.0053]


In [19]:
print("Macro-averaged F1-scores: ", f1_scores)
print("Mean MA F1-scores: ", round(np.mean(f1_scores), 4))
print("Std MA F1-scores: ", round(np.std(f1_scores), 4))

Macro-averaged F1-scores:  [0.2759, 0.2764, 0.2435, 0.2467, 0.2612, 0.2571, 0.2644, 0.2513, 0.2744, 0.2524]
Mean MA F1-scores:  0.2603
Std MA F1-scores:  0.0116


In [20]:
print("Macro-averaged MAE: ", mmae_scores)
print("Mean MMAE-scores: ", round(np.mean(mmae_scores), 4))
print("Std MMAE-scores: ", round(np.std(mmae_scores), 4))

Macro-averaged MAE:  [array([2.35875512]), array([2.06876628]), array([2.20102148]), array([2.55498926]), array([2.32392906]), array([2.45270823]), array([2.33132674]), array([2.21499986]), array([2.24085722]), array([2.31363625])]
Mean MMAE-scores:  2.3061
Std MMAE-scores:  0.1293
